# Export CosPlace and EigenPlace Models as ONNX with Dynamic Axes

The models `CosPlace` and `EigenPlace` are exported to ONNX format with dynamic axes for batch size and image size. This allows the models to handle variable input sizes during inference.

The repositories for these models can be found at:
- [CosPlace](https://github.com/gmberton/CosPlace/tree/main)
- [EigenPlace](https://github.com/gmberton/EigenPlaces)

## Imports and Helpers

These functions are used to load and export the models in ONNX format with dynamic axes.

In [ ]:
import torch
import sys
import os
import cv2
import numpy as np
import onnxruntime as ort


def add_to_path(folder):
    """Add EigenPlaces/CosPlace directory to Python path."""
    cosplace_path = os.path.abspath(folder)
    if cosplace_path not in sys.path:
        sys.path.insert(0, cosplace_path)

def load_image(image_path, input_size=(224, 224), mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)):
    """Load and preprocess an image."""
    img = cv2.imread(image_path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img = cv2.resize(img, input_size)
    img = img.astype(np.float32) / 255.0
    img = (img - mean) / std
    img = np.transpose(img, (2, 0, 1))  # HWC to CHW
    img = np.expand_dims(img, axis=0)  # Add batch dim
    img = img.astype(np.float32)
    return img

def infer_onnx(model_path, img, use_gpu=True):
    """Run inference using ONNX model."""
    # --- Run ONNX inference ---
    providers = ['CUDAExecutionProvider', 'CPUExecutionProvider'] if use_gpu else ['CPUExecutionProvider']
    session = ort.InferenceSession(model_path, providers=providers)

    input_name = session.get_inputs()[0].name
    output_name = session.get_outputs()[0].name

    output = session.run([output_name], {input_name: img})[0]

    # --- Post-process ---
    output = np.array(output)
    print("Output shape:", output.shape)
    return output

def infer_torch(model, img):
    """Run inference using PyTorch model."""
    model.eval()
    with torch.no_grad():
        torch_input = torch.from_numpy(img).float()
        torch_output = model(torch_input).cpu().numpy()
    print("Output shape:", torch_output.shape)
    return torch_output

def print_top5(output):
    """Print top-5 predictions."""
    top5_indices = np.argsort(output[0])[::-1][:5]
    top5_scores = output[0][top5_indices]

    print("Top-5 scores:")
    for i, (idx, score) in enumerate(zip(top5_indices, top5_scores)):
        print(f"{i+1}: Class {idx} — Score: {score:.4f}")

def compare_outputs(torch_output, onnx_output):
    """Compare outputs from PyTorch and ONNX models."""
    diff = np.abs(torch_output - onnx_output).max()
    print("Max difference between PyTorch and ONNX outputs:", diff)

torch.version.__version__  # Ensure PyTorch is installed and check version


In [ ]:
image_path = "img.png"  # Path to your test image
img = load_image(image_path, input_size=(1024,1024))  # Load and preprocess the image
img.shape  # Check the shape of the image

### EigenPlaces

We start by loading a pretrained eigenplaces model and running inference on a test image.

In [ ]:
model_original = torch.hub.load("gmberton/eigenplaces", "get_trained_model", backbone="ResNet18", fc_output_dim=512)

In [ ]:
output_original = infer_torch(model_original, img)  # Run inference with PyTorch model
print_top5(output_original)  # Print top-5 predictions

Next, we load the modified layers and reload the model with the new layers. This is necessary to ensure that the model can export dynamic input sizes. 

**IMPORTANT**: Restart the kernel at this point to ensure the model is loaded correctly.

In [ ]:
add_to_path("EigenPlaces")  # Add EigenPlaces directory to path
from hubconf import get_trained_model

model = get_trained_model(backbone="ResNet18", fc_output_dim=512)  # Load modified model

In [ ]:
output_modified = infer_torch(model, img)  # Run inference with PyTorch model
print_top5(output_modified)  # Print top-5 predictions

Finally, we can export the modified model to ONNX format with dynamic axes.

In [ ]:
model.eval()

dummy_input = torch.randn(1, 3, 224, 224)

torch.onnx.export(
    model,
    dummy_input,
    "ImageEmbedding_EigenPlace_ResNet18_512.onnx",
    export_params=True,
    opset_version=12,
    do_constant_folding=True,
    input_names=["input"],
    output_names=["output"],
    dynamic_axes={
        "input": {0: "batch_size", 2: "height", 3: "width"},
        "output": {0: "batch_size"}
    }
)

After we export the model, we can run inference on the ONNX model to verify that it works correctly.

In [ ]:
output_onnx = infer_onnx("ImageEmbedding_EigenPlace_ResNet18_512.onnx", img)  # Run inference on ONNX model
print_top5(output_onnx)  # Print top-5 predictions

While we're at it we can also export a fixed sized model.

In [ ]:
model.eval()

dummy_input = torch.randn(1, 3, 512, 512)

torch.onnx.export(
    model,
    dummy_input,
    "ImageEmbedding_EigenPlace_ResNet18_512_512x512.onnx",
    export_params=True,
    opset_version=12,
    do_constant_folding=True,
    input_names=["input"],
    output_names=["output"],
    dynamic_axes={
        "input": {0: "batch_size"},
        "output": {0: "batch_size"}
    }
)

## CosPlace

As above, load the official CosPlace model and run inference on a test image.

In [ ]:
model_original = torch.hub.load("gmberton/cosplace", "get_trained_model", backbone="ResNet18", fc_output_dim=32)

In [ ]:
output_original = infer_torch(model_original, img)  # Run inference with PyTorch model
print_top5(output_original)  # Print top-5 predictions

Next, load the modified layers. Again, this is necessary to ensure that the model can export dynamic input sizes.

**IMPORTANT** You must restart the kernel at this point to ensure the model is loaded correctly.

Finally, we can export the modified model to ONNX format with dynamic axes.

In [ ]:
add_to_path("CosPlace")  # Add CosPlace directory to path
from hubconf import get_trained_model

model = get_trained_model(backbone="ResNet18", fc_output_dim=32)  # Load modified model

In [ ]:
output_modified = infer_torch(model, img)  # Run inference with PyTorch model
print_top5(output_modified)  # Print top-5 predictions

Finally, we can export the model to ONNX format with dynamic axes for batch size and image size.

In [ ]:
model.eval()

dummy_input = torch.randn(1, 3, 224, 224)

torch.onnx.export(
    model,
    dummy_input,
    "ImageEmbedding_CosPlace_ResNet18_32.onnx",
    export_params=True,
    opset_version=12,
    do_constant_folding=True,
    input_names=["input"],
    output_names=["output"],
    dynamic_axes={
        "input": {0: "batch_size", 2: "height", 3: "width"},
        "output": {0: "batch_size"}
    }
)

After exporting, we can run inference on the ONNX model to verify that it works correctly.

In [ ]:
output_onnx = infer_onnx("ImageEmbedding_CosPlace_ResNet18_32.onnx", img)  # Run inference on ONNX model
print_top5(output_onnx)  # Print top-5 predictions

While we're at it we can also export a fixed sized model.

In [ ]:
model.eval()

dummy_input = torch.randn(1, 3, 512, 512)

torch.onnx.export(
    model,
    dummy_input,
    "ImageEmbedding_CosPlace_ResNet18_32_512x512.onnx",
    export_params=True,
    opset_version=12,
    do_constant_folding=True,
    input_names=["input"],
    output_names=["output"],
    dynamic_axes={
        "input": {0: "batch_size"},
        "output": {0: "batch_size"}
    }
)